### Figure 1
- schematic
- spectrogram for NSD and SD
- percentage sleep fraction during first 5h of POST for both SD and NSD sessions


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
import pandas as pd
import seaborn as sns
from neuropy.plotting import Fig
from scipy import stats
from subjects import epoch_span,boxplot_kw, light_cycle_span,boxplot_kw
from neuropy.plotting.dataviz import boxplot

plt.rcParams["font.family"] = "cursive"
grpdata = subjects.GroupData()
group = ["NSD", "SD"]

#SWA
swa,swa_time,swa_states = grpdata.swa_examples.values()
# brainstates 
states = grpdata.brainstates_proportion

#frate
frate_ratio = grpdata.frate_ratio_nsd_vs_sd
frate = grpdata.frate_zscore
frate = frate[frate.bins> -4]
frate_inter = grpdata.frate_interneuron_around_Zt5
frate_change = grpdata.frate_change_1vs5
frate_change_pre_post = grpdata.frate_change_pre_to_post
colors_frate_change = subjects.colors_sd(1.2)

frate_quantiles = grpdata.frate_pre_to_maze_quantiles_in_POST
# frate_quantiles_shuffled= grpdata.frate_pre_to_maze_quantiles_in_POST_shuffled

# plt.clf()
fig = Fig(grid=(6,8),fontsize=8)
filename = subjects.figpath_sd / "figure1_font"

caption = "Figure 1: Changes to firing rate across SD and NSD sessions. \n(A) Slow wave amplitude during POST of example SD and NSD sessions from one animal.\n(B) Porportion time spent in each vigilance state in first 5 h of POST pooled across SD (left) and NSD (right).\n(C) Mean firing rates across across session pooled across animals. Light/dark cycles and sessions epochs are indicated above x-axis.\n(D) Percentage change in firing rate from 1st to 5th hour POST.\n(E) Pentiles firing rate ratio plotted in first 5h of POST. Pentiles were first calculated from percentage increase in firing rate from PRE to MAZE (increasing hue in color). Then mean firing rate of pentiles normalized by each pentile's firing rate in first time bin. Error bars indicate SEM." 


### SWA

In [ ]:
from neuropy.plotting import plot_epochs

subfig = fig.fig.add_subfigure(fig.gs[0, :4])
axs = subfig.subplots(4, 1, gridspec_kw=dict(height_ratios=[1, 4, 1, 4]))
# axs[0].plot([1,2,3],[1,2,3])
axs = axs.reshape(2, 2).T
for i in range(2):
    time = swa_time[i]

    plot_epochs(
        swa_states[i],
        ax=axs[0, i],
        labels_order=["nrem", "rem", "quiet", "active"],
        colors=subjects.colors_sleep_old,
    )
    axs[0,i].set_xlim(time[0],time[-1])
    axs[0, i].axis("off")

    axs[1, i].plot(time, swa[i],color=subjects.colors_sd(1.4)[i])
    # axs[1, i].fill_between(time, swa[i],ec=subjects.colors_sd(1.5)[i],color=subjects.colors_sd(1.5)[i],alpha=0.6)
    axs[1,i].set_xlim(time[0],time[-1])
    if i ==0:
        axs[1,i].set_xticks([])
    if i==1:
        xticks = np.arange(9)*3600+ time[0] 
        axs[1, i].set_xticks(xticks, np.arange(9))


### sleep fraction

In [ ]:
ax = fig.subplot(fig.gs[0, 4])
hue = ["active", "quiet", "rem", "nrem"]
color = list(subjects.colors_sleep.values())
ax = sns.boxplot(
    data=states,
    x="grp",
    y="prop",
    hue="state",
    ax=ax,
    hue_order=hue,
    palette=color,
    showfliers=False,
    linewidth=1,
    # boxprops=dict(edgecolor='none'),
    # showcaps=False,
    # medianprops=dict(color='',lw=2),
    # capsize=0.05,
    # legend= None,
)
# for i,artist in enumerate(ax.artists):
#     # Set the linecolor on the artist to the facecolor, and set the facecolor to None
#     col = artist.get_facecolor()
#     artist.set_edgecolor(col)
#     artist.set_facecolor('None')

#     # Each box has 6 associated Line2D objects (to make the whiskers, fliers, etc.)
#     # Loop over them here, and use the same colour as above
#     for j in range(i*6,i*6+6):
#         line = ax2.lines[j]
#         line.set_color(col)
#         line.set_mfc(col)
#         line.set_mec(col)
ax.legend([],frameon=False)
fig.legend(ax,subjects.colors_sleep.keys(),subjects.colors_sleep.values(),x=0.5)
ax.set_ylabel("State proportion")
ax.set_xlabel('')
ax.set_title("First 5 h of POST ")
fig.panel_label(ax=ax, label="B")


### zscored firing rate across session

### Quantify change from PRE(baseline) to POST

In [ ]:
from statannotations.Annotator import Annotator

# ax_1v5 = fig.subplot(fig.gs[2, 0])
subfig = fig.add_subfigure(fig.gs[3,:2])
axs = subfig.subplots(1,2)

for i,n in enumerate(['pyr','inter']):
    df = frate_change_pre_post[frate_change_pre_post['neuron_type']==n]
    sns.boxplot(
        data=df,
        x="zt",
        y="frate_change",
        hue="grp",
        palette=colors_frate_change,
        ax=axs[i],
        hue_order=group,
        dodge=True,
        # width=0.5,
        # **boxplot_kw(subjects.colors_sd(1.2)[i])
        # showfliers=False,
    )

    axs[i].legend("", frameon=False)
    axs[i].set_ylabel('')
    axs[i].tick_params('x',labelrotation=45)



### pre-to-maze quantiles firing rate

### saving

In [ ]:
fig.savefig(subjects.figpath_sd/'figure1',caption=caption)